In [37]:
from numba import cuda, jit
import numpy as np
import dataReader
from similarityMatrix import *

In [38]:
@cuda.jit
def kernel_get_average_and_norm(R, cols, rows, avg, norm):
    # row: n_users, col: n_items
    bix = cuda.blockIdx.x
    bdx = cuda.blockDim.x
    tix = cuda.threadIdx.x

    # thread ID -> item index
    # 1 block -> 1 item
    tid = bix * bdx + tix
    if tid < cols:
        countNonZero = 0
        sum = 0
        avgThread = 0
        
        # With per user: 
        for i in range(rows):
            # Who is the user that rated this item

            if R[i*cols + tid] > 0:
                sum += R[i*cols + tid]
                countNonZero += 1
            
        if countNonZero > 0:
            avgThread = sum / countNonZero
        else:
            avgThread = 0
        
        if (tid < cols):
            avg[tid] = avgThread

        sum = 0
        for i in range(rows):
            if (R[i*cols + tid] > 0):
                sum += (R[i*cols + tid] - avgThread)**2
        
        if (tid < cols):
            norm[tid] = sum

In [41]:
if __name__ == "__main__":
    data = dataReader.DataReader('../../data/ml-100k/ua.base').readTXT()
    utility_matrix = get_utility_matrix(data)

    # Step 1: Calculate the mean matrix by kernel 
    start_time = get_time()
    n_users = utility_matrix.shape[0]
    n_items = utility_matrix.shape[1]
    R = utility_matrix.ravel()

    avg = np.zeros(n_items)
    norm = np.zeros(n_items)
    block_size = 256
    grid_size = (n_items + block_size - 1) // block_size
    kernel_get_average_and_norm[grid_size, block_size](R, n_items, n_users, avg, norm)
    end_time = get_time()
    print("Time to calculate the mean matrix: ", end_time - start_time)

    print(avg)
    # np.savetxt('../../out/avg_kernel.csv', avg, delimiter=',')
    # np.savetxt('../../out/norm_kernel.csv', norm, delimiter=',')
    print("Done!")

Time to calculate the mean matrix:  0:00:00.016977
[3.85969388 3.19834711 3.05882353 ... 2.         3.         3.        ]
Done!


In [40]:
def change(a:np.array):
    a[0]=1

a =np.array([0,1,2,3,4])
print(a)
change(a)
print(a)

[0 1 2 3 4]
[1 1 2 3 4]
